In [38]:
import pandas as pd
import talib

In [39]:
daily = pd.read_excel('gbp_usd_1D.xlsx')
h1 = pd.read_excel('gbp_usd_1h.xlsx')

In [40]:
h1['atr'] = talib.ATR(
    close=h1['close'],
    high=h1['high'],
    low=h1['low'],
    timeperiod=24
)

In [41]:
h1['rsi'] = talib.RSI(
    real=h1['close'],
    timeperiod=9
)

In [42]:
h1['ma'] = talib.MA(
    real=h1['close'],
    timeperiod=24
)

In [43]:
h1['true_range'] = talib.TRANGE(
    close=h1['close'],
    high=h1['high'],
    low=h1['low'],
)

In [44]:
h1['range'] = h1['high'] - h1['low']

In [45]:
def candle_color(r):
    if r['open'] > r['close']:
        return 'r'
    if r['open'] < r['close']:
        return 'g'

    return 'y'


h1['candle_color'] = h1.apply(candle_color, axis=1)

In [46]:
def candle_type(r):
    ratio = r['true_range'] / r['atr']

    if ratio < 0.8:
        return 'spinning'
    if 1.2 > ratio >= 0.8:
        return 'standard'
    if 2.4 > ratio >= 1.2:
        return 'long'
    if ratio >= 2.4:
        return 'spike'


h1['candle_type'] = h1.apply(candle_type, axis=1)

In [47]:
h1 = h1.iloc[24:]

In [48]:
h1 = h1.drop(columns=['Unnamed: 0'])

In [49]:
h1 = h1.reset_index(drop=True)

In [51]:
def set_ma_status(row):
    if row['close'] > row['ma']:
        return 'upper'
    if row['close'] < row['ma']:
        return 'under'
    return 'eq'


h1['ma_status'] = h1.apply(set_ma_status, axis=1)

In [52]:
h1: pd.DataFrame = pd.DataFrame(h1)

In [60]:
h1['signal'] = pd.NA

In [61]:
ma_status = h1.at[0, 'ma_status']
for i, element in h1.iterrows():

    if ma_status == element['ma_status']:
        continue

    ma_status = element['ma_status']

    h1.loc[i, 'signal'] = 'buy' if ma_status == 'upper' else 'sell'



In [69]:
from dataclasses import dataclass,asdict


@dataclass
class TradeSignal:
    order_type: str
    entry_price: float
    sl: float
    tp0: float
    tp1: float
    tp2: float


signals = []
for i, element in h1.iterrows():
    
    if pd.isna(element['signal']):
        continue
    
    if element['signal'] == 'buy':
        signals.append(
            TradeSignal(
                order_type=element['signal'],
                entry_price=element['close'],
                sl=element['close'] - element['atr'],
                tp0=element['close'] + element['atr'] * 0.8,
                tp1=element['close'] + element['atr'] * 1.6,
                tp2=element['close'] + element['atr'] * 2.4,
            )
        )
        if element['signal'] == 'sell':
            signals.append(
                TradeSignal(
                    order_type=element['signal'],
                    entry_price=element['close'],
                    sl=element['close'] + element['atr'],
                    tp0=element['close'] - element['atr'] * 0.8,
                    tp1=element['close'] - element['atr'] * 1.6,
                    tp2=element['close'] - element['atr'] * 2.4,
                )
            )



In [72]:
trade_signals = pd.DataFrame([asdict(i) for i in signals])

In [73]:
trade_signals

,order_type,entry_price,sl,tp0,tp1,tp2
0,buy,1.30818,1.306327,1.309662,1.311144,1.312626
1,buy,1.31046,1.308475,1.312048,1.313636,1.315223
2,buy,1.31875,1.316868,1.320256,1.321761,1.323267
3,buy,1.31287,1.311069,1.314311,1.315751,1.317192
4,buy,1.31240,1.310073,1.314262,1.316124,1.317985
...,...,...,...,...,...,...
1914,buy,1.34310,1.341648,1.344261,1.345423,1.346584
1915,buy,1.34366,1.342241,1.344795,1.345931,1.347066
1916,buy,1.34325,1.341906,1.344326,1.345401,1.346477
1917,buy,1.34343,1.342134,1.344467,1.345503,1.346540
